In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

# remove display limits
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
df_path = Path("/home/yangzhe/data/binance/data/futures/um/daily/klines/BTCUSDT/1m/merge.feather")
df = pd.read_feather(df_path)
df["time"] = pd.to_datetime(df["Open time"], unit="ms")

In [3]:
df_out = pd.DataFrame()
df_out["time"] = df["time"]
df_out["open"] = df["Open"]
df_out["high"] = df["High"]
df_out["low"] = df["Low"]
df_out["close"] = df["Close"]
df_out["vol"] = df["Volume"]
df_out["vol_taker_buy"] = df["Taker buy base asset volume"]
df_out["vol_taker_sell"] = df_out["vol"] - df_out["vol_taker_buy"]
df_out["vol_taker_buy_ratio"] = df_out["vol_taker_buy"] / df_out["vol"]
df_out["vol_taker_sell_ratio"] = df_out["vol_taker_sell"] / df_out["vol"]

# sort by time
df_out = df_out.sort_values(by="time")
df_out.head()

,time,open,high,low,close,vol,vol_taker_buy,vol_taker_sell,vol_taker_buy_ratio,vol_taker_sell_ratio
0,2021-01-01 00:02:00,29036.41,29036.97,28993.19,29016.23,102.675,44.060,58.615,0.429121,0.570879
1,2021-01-01 00:03:00,29016.23,29023.87,28995.50,29002.92,85.762,56.768,28.994,0.661925,0.338075
2,2021-01-01 00:04:00,29005.93,29019.98,29000.00,29000.00,70.246,46.645,23.601,0.664024,0.335976
3,2021-01-01 00:05:00,29000.01,29000.01,28950.67,28965.02,121.190,23.646,97.544,0.195115,0.804885
4,2021-01-01 00:06:00,28963.07,28986.57,28962.40,28970.04,66.914,20.081,46.833,0.300102,0.699898


In [4]:
agg_dict = {
        "high": "max",
        "low": "min",
        "close": ["last", "mean", "std", "median", "skew"],
        "vol": ["sum", "mean", "std", "median", "skew"],
        "vol_taker_buy": "sum",
        "vol_taker_sell": "sum",
        "price_ema_60": "last",
        "vol_taker_buy_ratio": ["last", "mean", "std", "median", "skew"],
        "vol_taker_sell_ratio": ["last", "mean", "std", "median", "skew"],
    }

# get time accurate to hour
df_out["time_H"] = df_out["time"].dt.floor("H")
for hl in [5, 30, 60, 240, 1440]:
    df_out[f"price_ema_{hl}"] = df_out["close"].ewm(span=hl).mean()
    agg_dict[f"price_ema_{hl}"] = "last"
# group by hour
df_out_agg = df_out.groupby("time_H").agg(agg_dict)
# shift by 1 (don't use data at current Hour)
df_out_agg = df_out_agg.shift(1)
# flatten multi-index
df_out_agg.columns = ["__".join(x) for x in df_out_agg.columns.ravel()]
df_out_agg.reset_index(inplace=True)
df_out_agg.head()

/tmp/ipykernel_35332/1747149851.py:23: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df_out_agg.columns = ["__".join(x) for x in df_out_agg.columns.ravel()]


,time_H,high__max,low__min,close__last,close__mean,close__std,close__median,close__skew,vol__sum,vol__mean,vol__std,vol__median,vol__skew,vol_taker_buy__sum,vol_taker_sell__sum,price_ema_60__last,vol_taker_buy_ratio__last,vol_taker_buy_ratio__mean,vol_taker_buy_ratio__std,vol_taker_buy_ratio__median,vol_taker_buy_ratio__skew,vol_taker_sell_ratio__last,vol_taker_sell_ratio__mean,vol_taker_sell_ratio__std,vol_taker_sell_ratio__median,vol_taker_sell_ratio__skew,price_ema_5__last,price_ema_30__last,price_ema_240__last,price_ema_1440__last
0,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,29055.00,28706.00,29015.00,28914.487931,68.318759,28926.060,-0.353375,7637.647,131.683569,133.499782,94.5515,3.414068,3689.753,3947.894,28936.594480,0.353211,0.512246,0.178721,0.515137,-0.141465,0.646789,0.487754,0.178721,0.484863,0.141465,29014.196450,28958.391350,28919.430472,28915.266844
2,2021-01-01 02:00:00,29546.42,28975.46,29448.40,29338.059667,135.159156,29391.110,-1.371138,19543.401,325.723350,395.871810,180.9715,3.809761,10386.352,9157.049,29330.975000,0.605835,0.497387,0.170827,0.484256,-0.032809,0.394165,0.502613,0.170827,0.515744,0.032809,29434.497462,29397.715155,29191.269520,29140.166013
3,2021-01-01 03:00:00,29500.00,29159.16,29237.06,29330.345667,64.652735,29327.145,0.327839,10049.720,167.495333,141.286947,120.8870,2.597611,4513.355,5536.365,29307.423992,0.627714,0.479721,0.181485,0.467499,-0.116301,0.372286,0.520279,0.181485,0.532501,0.116301,29238.531437,29287.494520,29258.367005,29209.142322
4,2021-01-01 04:00:00,29379.41,29174.95,29302.11,29282.323000,34.207700,29282.020,-0.012804,5579.452,92.990867,71.198140,64.0740,2.394396,2969.050,2610.402,29293.320025,0.266898,0.527730,0.195164,0.536006,0.097386,0.733102,0.472270,0.195164,0.463994,-0.097386,29316.428214,29299.656947,29270.200638,29230.019280


In [5]:
# convert float64 to float32
for col in df_out_agg.columns:
    if df_out_agg[col].dtype == "float64":
        df_out_agg[col] = df_out_agg[col].astype("float32")
df_out_agg.head()

# select time window
start_date = "2022-01-01"
end_date = "2022-09-01"
df_out_agg = df_out_agg[(df_out_agg["time_H"] >= start_date) & (df_out_agg["time_H"] < end_date)]
df_out_agg.head()

,time_H,high__max,low__min,close__last,close__mean,close__std,close__median,close__skew,vol__sum,vol__mean,vol__std,vol__median,vol__skew,vol_taker_buy__sum,vol_taker_sell__sum,price_ema_60__last,vol_taker_buy_ratio__last,vol_taker_buy_ratio__mean,vol_taker_buy_ratio__std,vol_taker_buy_ratio__median,vol_taker_buy_ratio__skew,vol_taker_sell_ratio__last,vol_taker_sell_ratio__mean,vol_taker_sell_ratio__std,vol_taker_sell_ratio__median,vol_taker_sell_ratio__skew,price_ema_5__last,price_ema_30__last,price_ema_240__last,price_ema_1440__last
8736,2022-01-01 00:00:00,46508.148438,46127.851562,46210.558594,46384.101562,58.792118,46398.445312,-1.655074,5416.187988,90.269798,89.678909,59.081501,3.348396,2656.731934,2759.456055,46359.300781,0.490763,0.488338,0.156838,0.478812,0.135713,0.509237,0.511662,0.156838,0.521188,-0.135713,46232.250000,46339.171875,46368.949219,47092.367188
8737,2022-01-01 01:00:00,46729.730469,46235.800781,46650.011719,46475.085938,142.456375,46398.589844,0.361129,8703.708008,147.520477,108.638397,114.634003,1.361932,4660.413086,4043.294922,46518.667969,0.236499,0.511849,0.136680,0.544767,-0.139701,0.763501,0.488151,0.136680,0.455233,0.139701,46661.144531,46593.523438,46416.996094,47044.027344
8738,2022-01-01 02:00:00,46944.558594,46570.339844,46766.781250,46762.652344,79.346275,46771.410156,-0.859544,7276.249023,121.270813,87.281006,90.138000,1.879154,3689.836914,3586.412109,46739.761719,0.530720,0.498422,0.144541,0.500001,0.037516,0.469280,0.501578,0.144541,0.499999,-0.037516,46769.730469,46772.210938,46554.699219,47021.593750
8739,2022-01-01 03:00:00,46917.871094,46709.441406,46796.140625,46812.250000,41.887104,46808.734375,0.387282,4478.712891,74.645218,47.185741,62.308498,2.384829,2103.344971,2375.367920,46795.234375,0.379047,0.468533,0.174879,0.473972,-0.034456,0.620953,0.531467,0.174879,0.526028,0.034456,46797.984375,46799.007812,46655.148438,47004.824219
8740,2022-01-01 04:00:00,46888.000000,46746.320312,46789.558594,46822.875000,31.577396,46820.855469,0.055581,4208.992188,70.149864,40.845280,56.588001,1.336987,1928.026978,2280.965088,46822.714844,0.184513,0.460668,0.179759,0.425858,0.153956,0.815487,0.539332,0.179759,0.574142,-0.153956,46810.417969,46826.984375,46721.695312,46990.296875


In [6]:
# modify column names
df_out_agg.rename(columns={"high__max": "price__high", "low__min": "price__min"}, inplace=True)
for col in df_out_agg.columns:
    if col.startswith("close__"):
        df_out_agg.rename(columns={col: col.replace("close__", "price__")}, inplace=True)
for col in df_out_agg.columns:
    if col.endswith("__last"):
        df_out_agg.rename(columns={col: col.replace("__last", "")}, inplace=True)
df_out_agg.head()

,time_H,price__high,price__min,price,price__mean,price__std,price__median,price__skew,vol__sum,vol__mean,vol__std,vol__median,vol__skew,vol_taker_buy__sum,vol_taker_sell__sum,price_ema_60,vol_taker_buy_ratio,vol_taker_buy_ratio__mean,vol_taker_buy_ratio__std,vol_taker_buy_ratio__median,vol_taker_buy_ratio__skew,vol_taker_sell_ratio,vol_taker_sell_ratio__mean,vol_taker_sell_ratio__std,vol_taker_sell_ratio__median,vol_taker_sell_ratio__skew,price_ema_5,price_ema_30,price_ema_240,price_ema_1440
8736,2022-01-01 00:00:00,46508.148438,46127.851562,46210.558594,46384.101562,58.792118,46398.445312,-1.655074,5416.187988,90.269798,89.678909,59.081501,3.348396,2656.731934,2759.456055,46359.300781,0.490763,0.488338,0.156838,0.478812,0.135713,0.509237,0.511662,0.156838,0.521188,-0.135713,46232.250000,46339.171875,46368.949219,47092.367188
8737,2022-01-01 01:00:00,46729.730469,46235.800781,46650.011719,46475.085938,142.456375,46398.589844,0.361129,8703.708008,147.520477,108.638397,114.634003,1.361932,4660.413086,4043.294922,46518.667969,0.236499,0.511849,0.136680,0.544767,-0.139701,0.763501,0.488151,0.136680,0.455233,0.139701,46661.144531,46593.523438,46416.996094,47044.027344
8738,2022-01-01 02:00:00,46944.558594,46570.339844,46766.781250,46762.652344,79.346275,46771.410156,-0.859544,7276.249023,121.270813,87.281006,90.138000,1.879154,3689.836914,3586.412109,46739.761719,0.530720,0.498422,0.144541,0.500001,0.037516,0.469280,0.501578,0.144541,0.499999,-0.037516,46769.730469,46772.210938,46554.699219,47021.593750
8739,2022-01-01 03:00:00,46917.871094,46709.441406,46796.140625,46812.250000,41.887104,46808.734375,0.387282,4478.712891,74.645218,47.185741,62.308498,2.384829,2103.344971,2375.367920,46795.234375,0.379047,0.468533,0.174879,0.473972,-0.034456,0.620953,0.531467,0.174879,0.526028,0.034456,46797.984375,46799.007812,46655.148438,47004.824219
8740,2022-01-01 04:00:00,46888.000000,46746.320312,46789.558594,46822.875000,31.577396,46820.855469,0.055581,4208.992188,70.149864,40.845280,56.588001,1.336987,1928.026978,2280.965088,46822.714844,0.184513,0.460668,0.179759,0.425858,0.153956,0.815487,0.539332,0.179759,0.574142,-0.153956,46810.417969,46826.984375,46721.695312,46990.296875


In [7]:
# price_ema diff with price
for hl in [5, 30, 60, 240, 1440]:
    df_out_agg[f"price_ema_{hl}_diff"] = df_out_agg[f"price_ema_{hl}"] - df_out_agg["price"]
df_out_agg.head()

,time_H,price__high,price__min,price,price__mean,price__std,price__median,price__skew,vol__sum,vol__mean,vol__std,vol__median,vol__skew,vol_taker_buy__sum,vol_taker_sell__sum,price_ema_60,vol_taker_buy_ratio,vol_taker_buy_ratio__mean,vol_taker_buy_ratio__std,vol_taker_buy_ratio__median,vol_taker_buy_ratio__skew,vol_taker_sell_ratio,vol_taker_sell_ratio__mean,vol_taker_sell_ratio__std,vol_taker_sell_ratio__median,vol_taker_sell_ratio__skew,price_ema_5,price_ema_30,price_ema_240,price_ema_1440,price_ema_5_diff,price_ema_30_diff,price_ema_60_diff,price_ema_240_diff,price_ema_1440_diff
8736,2022-01-01 00:00:00,46508.148438,46127.851562,46210.558594,46384.101562,58.792118,46398.445312,-1.655074,5416.187988,90.269798,89.678909,59.081501,3.348396,2656.731934,2759.456055,46359.300781,0.490763,0.488338,0.156838,0.478812,0.135713,0.509237,0.511662,0.156838,0.521188,-0.135713,46232.250000,46339.171875,46368.949219,47092.367188,21.691406,128.613281,148.742188,158.390625,881.808594
8737,2022-01-01 01:00:00,46729.730469,46235.800781,46650.011719,46475.085938,142.456375,46398.589844,0.361129,8703.708008,147.520477,108.638397,114.634003,1.361932,4660.413086,4043.294922,46518.667969,0.236499,0.511849,0.136680,0.544767,-0.139701,0.763501,0.488151,0.136680,0.455233,0.139701,46661.144531,46593.523438,46416.996094,47044.027344,11.132812,-56.488281,-131.343750,-233.015625,394.015625
8738,2022-01-01 02:00:00,46944.558594,46570.339844,46766.781250,46762.652344,79.346275,46771.410156,-0.859544,7276.249023,121.270813,87.281006,90.138000,1.879154,3689.836914,3586.412109,46739.761719,0.530720,0.498422,0.144541,0.500001,0.037516,0.469280,0.501578,0.144541,0.499999,-0.037516,46769.730469,46772.210938,46554.699219,47021.593750,2.949219,5.429688,-27.019531,-212.082031,254.812500
8739,2022-01-01 03:00:00,46917.871094,46709.441406,46796.140625,46812.250000,41.887104,46808.734375,0.387282,4478.712891,74.645218,47.185741,62.308498,2.384829,2103.344971,2375.367920,46795.234375,0.379047,0.468533,0.174879,0.473972,-0.034456,0.620953,0.531467,0.174879,0.526028,0.034456,46797.984375,46799.007812,46655.148438,47004.824219,1.843750,2.867188,-0.906250,-140.992188,208.683594
8740,2022-01-01 04:00:00,46888.000000,46746.320312,46789.558594,46822.875000,31.577396,46820.855469,0.055581,4208.992188,70.149864,40.845280,56.588001,1.336987,1928.026978,2280.965088,46822.714844,0.184513,0.460668,0.179759,0.425858,0.153956,0.815487,0.539332,0.179759,0.574142,-0.153956,46810.417969,46826.984375,46721.695312,46990.296875,20.859375,37.425781,33.156250,-67.863281,200.738281
